# Use Python function to predict user's satisfaction for car rental company with `ibm-watson-machine-learning`

This notebook trains a **Keras** (TensorFlow) model to predict customer satisfaction based on the feedback that has been provided. The notebook also demonstrates how you can use the **python function** for the deep learning model data preprocessing required before you start model scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.6 and the watson-machine-learning-client package.


Contents:
1. [Setup](#setup)
2. [Load and explore data](#load)
3. [Create Keras model using TensorFlow backend](#model)
4. [Store the model in the repository](#persistence)
5. [Deploy the model](#deployment)
6. [Score the model](#score)
7. [Define, store and deploy Python function](#ai_function)
8. [Clean up](#cleanup)
9. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [29]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

### Install TensorFlow


**Tip:** Restart the kernel (**Kernel** -> **Restart**)

<a id="load"></a>
## 2. Load and explore data

In this section you read in the `car_rental_training_data.csv` file, load it as a pandas dataFrame and then perform a basic exploration. 


In [127]:
!pip install wget  | tail -n 1

In [72]:
import wget

link_to_data = 'https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/cars-4-you/car_rental_training_data.csv'
filename = wget.download(link_to_data)

filename

'car_rental_training_data (1).csv'

Load the data as a pandas dataFrame.


In [73]:
import pandas as pd

data = pd.read_csv(filename, sep=';') 
data.head()


,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,83,Female,M,2,48.85,Inactive,Yes,I thought the representative handled the initi...,0,Product: Availability/Variety/Size,Free Upgrade
1,1307,Female,M,0,55.00,Inactive,No,I have had a few recent rentals that have take...,0,Product: Availability/Variety/Size,Voucher
2,1737,Male,M,0,42.35,Inactive,Yes,car cost more because I didn't pay when I rese...,0,Product: Availability/Variety/Size,Free Upgrade
3,3721,Male,M,2,61.71,Inactive,Yes,I didn't get the car I was told would be avail...,0,Product: Availability/Variety/Size,Free Upgrade
4,11,Male,S,2,56.47,Active,No,If there was not a desired vehicle available t...,1,Product: Availability/Variety/Size,NaN




**Note:** 0 - not satisfied, 1 - satisfied


Extract the required columns and count the number of records.


In [74]:
complain_data = data[['Customer_Service', 'Satisfaction']]

In [75]:
complain_data.count()

Customer_Service    486
Satisfaction        486
dtype: int64

<a id="model"></a>
## 3. Create a Keras model using the TensorFlow backend

In [76]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import os
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

### 3.1 Prepare the data


In [77]:
max_fatures = 500

for idx,row in complain_data.iterrows():
    row[0] = row[0].replace('rt',' ')

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(complain_data['Customer_Service'].values)
X = tokenizer.texts_to_sequences(complain_data['Customer_Service'].values)

maxlen = 50

X = pad_sequences(X, maxlen=maxlen)
X.shape

(486, 50)

### Split the data into train and test data sets.


In [78]:
Y = complain_data['Satisfaction'].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

X_train.shape,Y_train.shape
X_test.shape,Y_test.shape

((161, 50), (161,))

### 3.2 Design and train the model
Create the network definition based on the Gated Recurrent Unit (Cho et al. 2014).


In [ ]:

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(max_fatures, embedding_vector_length, input_length=maxlen))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 32)            16000     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 72,405
Trainable params: 72,405
Non-trainable params: 0
_________________________________________________________________


### Train the model.


In [81]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=64)

Train on 325 samples, validate on 161 samples
Epoch 1/20
325/325 [==============================] - 1s 4ms/step - loss: 0.6881 - acc: 0.5877 - val_loss: 0.6866 - val_acc: 0.5342
Epoch 2/20
325/325 [==============================] - 0s 422us/step - loss: 0.6748 - acc: 0.5785 - val_loss: 0.6799 - val_acc: 0.5342
Epoch 3/20
325/325 [==============================] - 0s 426us/step - loss: 0.6615 - acc: 0.5785 - val_loss: 0.6701 - val_acc: 0.5342
Epoch 4/20
325/325 [==============================] - 0s 399us/step - loss: 0.6397 - acc: 0.5785 - val_loss: 0.6603 - val_acc: 0.5342
Epoch 5/20
325/325 [==============================] - 0s 420us/step - loss: 0.6176 - acc: 0.6738 - val_loss: 0.6256 - val_acc: 0.7205
Epoch 6/20
325/325 [==============================] - 0s 439us/step - loss: 0.5701 - acc: 0.7815 - val_loss: 0.6573 - val_acc: 0.6832
Epoch 7/20
325/325 [==============================] - 0s 414us/step - loss: 0.5605 - acc: 0.7569 - val_loss: 0.5481 - val_acc: 0.7453
Epoch 8/20
325/325

In [82]:
("Best accuracy on test: %3.3f" % numpy.array(history.history['val_accuracy']).max())

'Best accuracy on test: 0.957'

**Note:** For purpose of this demo, model tuning has been skipped.

Store and archive the model in the notebook filesystem.



### Evaluate the model

In [83]:
scores = model.evaluate(X_test, Y_test, verbose=0)
("Evaluation Accuracy: %.2f%%" % (scores[1]*100))

'Evaluation Accuracy: 95.65%'

In [84]:
filename = 'complain_model.h5'
model.save(filename)

# Compress Keras model
tar_filename = filename + ".tgz"
cmdstring = "tar -zcvf " + tar_filename + " " + filename
score = tar_filename
os.system(cmdstring);

<a id="persistence"></a>
## 4. Store the model in the repository

In this section, you will learn how to store your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 4.1: Publish model
#### Publish model in Watson Machine Learning repository on Cloud.


Get software specification for tensorflow model.

In [85]:
software_spec_id = client.software_specifications.get_id_by_name('tensorflow_2.1-py3.7')
print(software_spec_id)

2b73a275-7cbf-420b-a912-eae7f436e0bc


Store model

In [87]:

metadata = {
    client.repository.ModelMetaNames.NAME: "CARS4U - Satisfaction Prediction Model",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.1',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_id
}
print(metadata)    
stored_model_details = client.repository.store_model(tar_filename, meta_props=metadata)

{'name': 'CARS4U - Satisfaction Prediction Model', 'type': 'keras_2.2.5', 'software_spec': '2b73a275-7cbf-420b-a912-eae7f436e0bc'}


### 4.2: Get model details

In [88]:
import json

model_uid = client.repository.get_model_uid(stored_model_details)

model_details = client.repository.get_details(model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "2b73a275-7cbf-420b-a912-eae7f436e0bc"
    },
    "type": "keras_2.2.5"
  },
  "metadata": {
    "created_at": "2020-08-12T07:54:33.970Z",
    "id": "2e7ad091-9f60-4205-a81d-ceaa623e2129",
    "modified_at": "2020-08-12T07:54:44.057Z",
    "name": "CARS4U - Satisfaction Prediction Model",
    "owner": "IBMid-550004P1CE",
    "space_id": "f98c8db8-0d02-4ba6-a873-f299e2dc2378"
  }
}


<a id="deployment"></a>
## 5. Deploy the model
In this section you will learn how to create batch deployment to create job using the Watson Machine Learning Client.

You can use commands below to create batch deployment for stored model (web service).

### 5.1: Create model deployment


In [89]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "CARS4U - Satisfaction Prediction Model Deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [90]:
deployment_details = client.deployments.create(model_uid, meta_props, background_mode=False)




#######################################################################################

Synchronous deployment creation for uid: '2e7ad091-9f60-4205-a81d-ceaa623e2129' started

#######################################################################################


initializing.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='816730ac-e4e1-4bba-b91f-ed016ffea150'
------------------------------------------------------------------------------------------------




**Note**: Here we use deployment url saved in published_model object. In next section, we show how to retrieve deployment url from Watson Machine Learning instance.



In [91]:
deployment_uid = client.deployments.get_uid(deployment_details)

Now, You can list all deployments.

In [128]:
client.deployments.list()

### 5.2 Get deployment details

In [94]:
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': '2e7ad091-9f60-4205-a81d-ceaa623e2129'},
  'custom': {},
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'S', 'num_nodes': 1},
  'name': 'CARS4U - Satisfaction Prediction Model Deployment',
  'online': {},
  'space_id': 'f98c8db8-0d02-4ba6-a873-f299e2dc2378',
  'status': {'online_url': {'url': 'https://wml-fvt.ml.test.cloud.ibm.com/ml/v4/deployments/816730ac-e4e1-4bba-b91f-ed016ffea150/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2020-08-12T07:54:50.139Z',
  'id': '816730ac-e4e1-4bba-b91f-ed016ffea150',
  'modified_at': '2020-08-12T07:54:50.139Z',
  'name': 'CARS4U - Satisfaction Prediction Model Deployment',
  'owner': 'IBMid-550004P1CE',
  'space_id': 'f98c8db8-0d02-4ba6-a873-f299e2dc2378'}}

<a id="score"></a>
## 6. Score the model
Let's see if our deployment works.


You can use below method to do test scoring request against deployed model.



In [95]:
deployment_id = client.deployments.get_id(deployment_details)

In [96]:
deployment_id

'816730ac-e4e1-4bba-b91f-ed016ffea150'

**Action**: Prepare scoring payload with records to score.

In [97]:
index = 5

scoring_data = X[index].tolist()
print(X_test[index])
print(Y_test[index])
 

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   4  43  19   3 242  25
 235   6  22 353   1 237  12  10   8 107  14  34   5  56]
1


In [98]:
scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA:
    [
        {
        'values': [scoring_data]
        }
   ]
}


In [99]:
scores = client.deployments.score(deployment_id, scoring_payload)

Let's print the scoring results.

In [100]:
print(str(scores))

{'predictions': [{'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[9.750197932589799e-05], [0], [9.750197932589799e-05]]]}]}


## 7. Define, store and deploy Python function
Let's define a function that does data preprocessing and model scoring. As we saw in the previous cells, the model expects numerical input, so the text comment needs to be preprocessed.

### 7.1 Define generic parameters

In [101]:
ai_params = {
    'deployment_id': deployment_id,
    'wml_credentials': wml_credentials,
    'word_index': tokenizer.word_index,
    'space_id': space_id
}

In [102]:
def score_generator(params=ai_params):
    import re
    from ibm_watson_machine_learning import APIClient
    client = APIClient(params['wml_credentials'])
    client.set.default_space(params['space_id'])
    def score(payload):
        max_fatures = 500
        maxlen = 50
        preprocessed_records = []
        complain_data = payload['input_data'][0]['values']
        word_index = params['word_index']
        for data in complain_data:
            comment = data[0]
            cleanString = re.sub(r"[!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~]", "", comment)
            splitted_comment = cleanString.split()[:maxlen]
            hashed_tokens = []
            for token in splitted_comment:
                index = word_index.get(token, 0)
                if index < 501 and index > 0:
                    hashed_tokens.append(index)
            hashed_tokens_size = len(hashed_tokens)
            padded_tokens = [0]*(maxlen-hashed_tokens_size) + hashed_tokens
            preprocessed_records.append(padded_tokens)
        scoring_payload = {'values': preprocessed_records}
        scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA: [scoring_payload]}
        return client.deployments.score(params['deployment_id'], scoring_payload)
    return score


#### Test the function locally


In [103]:
sample_payload = {
    "input_data": [
        {
            "fields": ["feedback"],
            "values": [['delayed shuttle, almost missed flight, bad customer service'],['The car was great and they were able to provide all features I wanted with limited time they had.']]
        }       
    ]
}

In [104]:
print(sample_payload)

{'input_data': [{'fields': ['feedback'], 'values': [['delayed shuttle, almost missed flight, bad customer service'], ['The car was great and they were able to provide all features I wanted with limited time they had.']]}]}


In [105]:
score = score_generator()
score_ai = score(sample_payload)
print(score_ai)

{'predictions': [{'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[0.0059976172633469105], [0], [0.0059976172633469105]], [[0.9992249011993408], [1], [0.9992249011993408]]]}]}


**Note:** 0 - not satisfied. 1 - satisfied

### 7.2 Store the function

In [106]:
software_spec_id = client.software_specifications.get_id_by_name("default_py3.7")
software_spec_id 

'0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda'

In [107]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'CARS4U - Satisfaction Prediction - AI Function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id,
}

function_details = client.repository.store_function(score_generator, meta_data)

In [108]:
client.repository.list_functions()

------------------------------------  ----------------------------------------------  ------------------------  ------
GUID                                  NAME                                            CREATED                   TYPE
6f12a788-6ebd-45a0-996e-8e5bdaa0aebd  CARS4U - Satisfaction Prediction - AI Function  2020-08-12T07:55:13.002Z  python
------------------------------------  ----------------------------------------------  ------------------------  ------


### 7.3 Deploy the function

In [109]:
function_uid = client.repository.get_function_uid(function_details)
published_function_id = client.repository.get_function_id(function_details)
client.repository.get_function_details(function_uid)

{'entity': {'software_spec': {'id': '0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda'},
  'type': 'python'},
 'metadata': {'created_at': '2020-08-12T07:55:13.236Z',
  'id': '6f12a788-6ebd-45a0-996e-8e5bdaa0aebd',
  'modified_at': '2020-08-12T07:55:16.254Z',
  'name': 'CARS4U - Satisfaction Prediction - AI Function',
  'owner': 'IBMid-550004P1CE',
  'space_id': 'f98c8db8-0d02-4ba6-a873-f299e2dc2378'}}

In [110]:
meta_data = {
    client.deployments.ConfigurationMetaNames.NAME: 'CARS4U - Satisfaction Prediction - AI Function Deployment',
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [111]:
function_deployment_details = client.deployments.create(published_function_id,meta_data)



#######################################################################################

Synchronous deployment creation for uid: '6f12a788-6ebd-45a0-996e-8e5bdaa0aebd' started

#######################################################################################


initializing....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b1588d96-609f-4c4c-9200-a2be5d276cf3'
------------------------------------------------------------------------------------------------




### 7.4 Score function

In [112]:
import json
print(function_deployment_details['metadata']['id'])
scoring_response = client.deployments.score(function_deployment_details['metadata']['id'], sample_payload)

print(json.dumps(scoring_response, indent=3))

b1588d96-609f-4c4c-9200-a2be5d276cf3
{
   "predictions": [
      {
         "fields": [
            "prediction",
            "prediction_classes",
            "probability"
         ],
         "values": [
            [
               [
                  0.0059976172633469105
               ],
               [
                  0
               ],
               [
                  0.0059976172633469105
               ]
            ],
            [
               [
                  0.9992249011993408
               ],
               [
                  1
               ],
               [
                  0.9992249011993408
               ]
            ]
         ]
      }
   ]
}


<a id="cleanup"></a>
## 8. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 9. Summary and next steps

You successfully completed this notebook! You learned how to use Keras machine learning library as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=analytics?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Wojciech Jargielo**, Software Engineer

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.

